In [1]:
from functions import *

# Schnorr Signatures and TapTweaks

### Reading
- [What the heck is Schnorr - Rajarshi Maitra]((https://medium.com/bitbees/what-the-heck-is-schnorr-52ef5dba289f)
- https://bitcoin.stackexchange.com/questions/107954/does-every-private-key-have-two-public-keys-ie-y-and-negated-y-secp256k1/107957#107957

Related reading
- https://github.com/t-bast/lightning-docs/blob/master/schnorr.md

The following material adapted from the Bitcoin Optech [Schnorr Taproot workshop](https://bitcoinops.org/en/schorr-taproot-workshop/).


## Introduction

You may have heard that taproot introduced schnorr signatures, and that taproot signatures have a linear property, but what does this mean? In this section we'll cover some basics of schnorr signatures, and how commitments can be encoded into public keys via taptweaks. By the end of this section you should understand the advantages of schnorr signatures over ECDSA signatures, and how to tweak public and private keys.


## Background

Since schnorr signatures were only recently introduced in bitcoin in taproot, you may think schnorr signatures were developed after ECDSA signatures, but you'd be wrong. Rajarshi Maitra puts it well in his blog post = [What the heck is Schnorr]((https://medium.com/bitbees/what-the-heck-is-schnorr-52ef5dba289f)

> Schnorr signature was invented by Claus-Peter Schnorr back in the 1980s. His digital signature implementation was much simpler than contemporary other algorithms. But he did the worst thing that can ever happen to any kind of knowledge. He patented it before he published the paper. Because of his patent, the Schnorr signature algorithm did not see any widespread use for decades. In fact, ECDSA’s predecessor, DSA was created specifically to bypass the Schnorr patent.

One of the ways ECDSA was designed to bypass the Schnorr patent was to break the useful linearity of Schnorr signatures. For this reason the math behind Schnorr signatures is also perhaps more intuitive than ECDSA. 

Elliptic curve private keys have a linear property, such that the sum of private keys will produce a sum of public keys equal to the sum of the individiual public keys. For a review of this principle, refer to the section '[Elliptic Curve Math Review](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/elliptic_curve_math_review.ipynb)'. With Schnorr signatures, this linearity extends to the signatures themselves. 

## Schnorr Signatures

### x-only pubkeys

Taproot uses a new pubkey format known as 'x-only pubkeys' as they contain only the x-coordinate of the public key point. Note that the public key point will always be interpreted as having an even y-coordinate. This means if the secret key produces a public key with an odd y-coordinate, the secret key and public key will need to be negated. For more on this see the 'x-only pubkey' section in the Elliptic Curve Math Review. Below we'll use the reference code from BIP340 for generating the pubkey from the secret key.

In [10]:
def pubkey_gen(seckey: bytes) -> bytes:
    d0 = tr.int_from_bytes(seckey)
    if not (1 <= d0 <= tr.n - 1):
        raise ValueError('The secret key must be an integer in the range 1..n-1.')
    P = tr.point_mul(tr.G, d0)
    assert P is not None
    return tr.bytes_from_point(P)

#### Recap of creating and verifying schnorr signatures

Below is a basic example of creating a private and public keypair, signing the message 'msg', then verifying it.

In [11]:
# Generate a key pair
privkey = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000001")
pubkey = pubkey_gen(privkey)
print(f"Private key: {privkey.hex()}\nPublic key: {pubkey.hex()}\n")

msg = sha256(b'msg')
aux_rand = bytes(32) # auxiliary random data
sig = tr.schnorr_sign(msg, privkey, aux_rand)
print(f"Signature: {sig.hex()}\n")

assert(tr.schnorr_verify(msg, pubkey, sig))
print("Success!")

Private key: 0000000000000000000000000000000000000000000000000000000000000001
Public key: 79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798

Signature: 4791b03f4381ddd5f22847d1c736e64d1d8d02f90af83a1373bb6392aae8d132660cc589d47e13a40c6bce2a408887d56f7ecac3895b9a9e5d75fe78b6f2478e

Success!


## TapTweaks

* Part 1: Tweaking the public key; commitment schemes with tweaks
* Part 2: Spending a (tweaked) taproot output along the key path
* Part 3 (Case Study): contract commitments

### What is a 'taptweak'?

The linear property of BIP340 means that we can encode a commitment into a public key, and then reveal that commitment when signing with the private key. We do that by tweaking the private key with the commitment, and using the associated tweaked pubkey. When signing, we can reveal that the original keys were tweaked by the commitment.

In part 1, we'll learn about how private/public key pairs can be tweaked, and how we can use that to create a secure commitment scheme. In part 2, we'll create a segwit v1 output and spend it along the key path, using a tweaked private and public key. Part 3 of this chapter is a case study, showing how pay-to-contract with tweaked keys can be used instead of OP_RETURN outputs to create timestamped commitments.

## Part 1: Tweaking the public key

Instead of using our original public key as the witness program, we use a tweaked public key.

* `[01] [32B Tweaked Public Key]`

Tweaking a public key means to alter it with a value (the tweak) such that it remains spendable with knowledge of the original private key and tweak.

* `x` / `P`: Original private key / public key pair
* `t` / `T`: Tweak value / tweak point
* Output script: `[01] [P + T]` = `[01] [xG + tG]`
* Spendable by the tweaked private key: `x + t`

An observer cannot distinguish between a tweaked and untweaked public key.

#### Example: Signing with a tweaked keypair

In this example, we generate a key pair as before, and then tweak both the private key and public key. We then sign with the tweaked private key and verify that the signature is valid.

A _tweak_ is positive scalar value `t` where `0 < t < SECP256K1_ORDER`. There is an associated tweak point `T` such that `T = t*G`.

The private key is tweaked by the tweak scalar: `x' = x + t` and the public key is tweaked by the tweak point: `P' = P + T`.

The tweaked private key `x'` can be used to produce a valid signature for the tweaked pubkey `P'`.

In [12]:
# Generate a key pair
privkey = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000001")
pubkey = pubkey_gen(privkey)
print(f"Private key: {privkey.hex()}\nPublic key: {pubkey.hex()}\n")

# Let's set a random tweak scalar 0 < t < SECP256K1_ORDER and derive its associated tweak point
tweak_val = bytes.fromhex("73fbb916db668bf8d6be37c171403793a890f28a10de7a3da40afe12ef5a4494")

# Generate the tweak point in the same way we'd generate a pubkey from a private key
tweak_point = pubkey_gen(tweak_val)
print(f"Tweak scalar: {tweak_val.hex()}\nTweak point: {tweak_point.hex()}\n")

# Derive the tweaked private key and public key
# Note that the private key may need to be negated to ensure the corresponding public key has
# an even Y-coordinate (BIP340)
d0 = int.from_bytes(privkey, byteorder="big")
P = tr.point_mul(tr.G, d0)
d = d0 if tr.has_even_y(P) else tr.SECP256K1_ORDER - d0
tweak_int = int.from_bytes(tweak_val, byteorder="big")
privkey_tweaked_int = (d + tweak_int) % tr.SECP256K1_ORDER

# Convert the tweaked private key back to bytes
privkey_tweaked = privkey_tweaked_int.to_bytes(32, "big")

# Generate the corresponding tweaked public key in the usual way
pubkey_tweaked = tr.pubkey_gen(privkey_tweaked)

print(f"privkey_tweaked: {privkey_tweaked.hex()}\npubkey_tweaked: {pubkey_tweaked.hex()}\n")

# Sign the message with tweaked key pair and verify the signature
msg = sha256(b'msg')
aux = bytes(32)
sig = tr.schnorr_sign(msg, privkey_tweaked, aux_rand)

assert(tr.schnorr_verify(msg, pubkey_tweaked, sig))
print("Success!")

Private key: 0000000000000000000000000000000000000000000000000000000000000001
Public key: 79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798

Tweak scalar: 73fbb916db668bf8d6be37c171403793a890f28a10de7a3da40afe12ef5a4494
Tweak point: 2526c792e88c0130779ca676e56f1c47e197b53307ecfb35e526238b1bfee43d

privkey_tweaked: 73fbb916db668bf8d6be37c171403793a890f28a10de7a3da40afe12ef5a4495
pubkey_tweaked: 6cd9429c131128c0d19a26699521c11a75923deb307eb5a4fe04d243082882bb

Success!


### TODO: Show why this commitment is not secure

In [13]:
def taproot_tweak_seckey(seckey0, h):
    seckey0 = tr.int_from_bytes(seckey0)
    P = tr.point_mul(tr.G, seckey0)
    seckey = seckey0 if tr.has_even_y(P) else tr.SECP256K1_ORDER - seckey0
    t = tr.int_from_bytes(tr.tagged_hash("TapTweak", tr.bytes_from_int(tr.x(P)) + h))
    if t >= tr.SECP256K1_ORDER:
        raise ValueError
    return tr.bytes_from_int((seckey + t) % tr.SECP256K1_ORDER)

In [14]:
def tweak_seckey(seckey, t):
    seckey = tr.int_from_bytes(seckey)
    t = tr.int_from_bytes(t)
    return tr.bytes_from_int((seckey + t) % tr.SECP256K1_ORDER)

In [15]:
tweak_seckey(privkey, tweak_val).hex()

'73fbb916db668bf8d6be37c171403793a890f28a10de7a3da40afe12ef5a4495'

## Commitment schemes with tweaks

Taproot uses the tweak as a commitment for spending script paths. However, simply applying the committed value as a public key tweak is not sufficient, as this does not represent a secure cryptographic commitment.

![test](images/taproot-workshop/taptweak0.jpg)

Instead, the committed value must first be hashed with the untweaked public key point. This commitment scheme is called *pay-to-contract*. **It does not allow the modification of a committed value for a given public key point Q.**

#### Example 2.2.5 - Pay-to-contract: Tweaking the pubkey with `H(P|msg)`

In this example, we demonstrate a _secure_ commitment scheme called pay-to-contract. The private key is tweaked with the scalar `H(P|c)`. Since `P` appears both inside and outside the hash, it isn't possible to solve for a different contract `c` by modifying `x`.

In [19]:
# Generate a key pair
privkey = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000004")

# Set the private key to an integer for elliptic curve operations
d0 = int.from_bytes(privkey, byteorder="big")

# We need this step to make sure we negate pubkeys with odd y-coordinates
d = d0 if tr.has_even_y(pubkey) else tr.SECP256K1_ORDER - d0

# Generate the public key. Note that we don't use the function pubkey_gen as
# we want the full point (not just the x-coordinate)
P = tr.point_mul(tr.G, d)

# The contract data we want to commit to
contract = "Alice agrees to pay 10 BTC to Bob"

# The tweak will commit to the contract data as well as the original pubkey to protect
# against the vulnerability mentioned in the previous part
tweak_int = tr.int_from_bytes(tr.tagged_hash("TapTweak", tr.bytes_from_int(tr.x(pubkey)) + contract.encode('utf-8')))

# Generate the tweak point
T = tr.point_mul(tr.G, tweak_int)

# Generate the tweaked pubkey by adding the tweak point and pubkey
TW = tr.point_add(P, T)

# Extract the x-coordinate for our final tweaked x-only pubkey
tweaked_x_only_pubkey = tr.bytes_from_int(tr.x(TW))

# We can check that we get the same pubkey when generating they pubkey using the tweaked secret key
pubkey_from_tweaked_privkey = pubkey_gen(tr.bytes_from_int((d + tweak_int) % tr.SECP256K1_ORDER))

assert(tweaked_x_only_pubkey == pubkey_from_tweaked_privkey)
print("Tweaked pubkey generated using tweaked private key: ", tweaked_x_only_pubkey.hex())
print("Tweaked pubkey generated without private key: ", pubkey_from_tweaked_privkey.hex())
print("Success!")

Tweaked pubkey generated using tweaked private key:  bba324ab8b1759427e62b8f2a1a15ac981748dec744b08c400efd25dc13e16a9
Tweaked pubkey generated without private key:  bba324ab8b1759427e62b8f2a1a15ac981748dec744b08c400efd25dc13e16a9
Success!
